<a href="https://colab.research.google.com/github/pharringtonp19/rfp/blob/main/notebooks/pld.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import jax 
import jax.numpy as jnp 

In [25]:
def sample(key, weights, features, clusters):
  subkey1, subkey2, subkey3, subkey4 = jax.random.split(key, 4)
  controls = jax.random.normal(subkey1, shape=(features,))
  time_period = jax.random.bernoulli(subkey2, shape=())
  cluster = jax.random.choice(subkey3, clusters, shape=())
  treatment = (cluster % 2 == 0.).astype(jnp.float32)
  outcome = jnp.dot(weights, controls) + treatment + time_period + cluster + 2*treatment*time_period
  return outcome, treatment, time_period, controls, jax.nn.one_hot(cluster, clusters)

In [33]:
features = 5 
clusters = 5
n = 1000
weights = jax.random.normal(jax.random.PRNGKey(0), shape=(features,))

In [34]:
y, d, t, f, c = jax.vmap(lambda key: sample(key, weights, features, clusters))(jax.random.split(jax.random.PRNGKey(0), n))
y, d, t = y.reshape(-1,1), d.reshape(-1,1), t.reshape(-1,1)

In [38]:
regs = jnp.hstack((d*t, d, jnp.ones_like(d), t, f, c))

In [39]:
ols = jnp.linalg.lstsq(regs, y)[0][0]

In [40]:
ols

DeviceArray([2.000001], dtype=float32)

In [57]:
left_reg = jnp.hstack((f,c, t))
left_coef = jnp.linalg.lstsq(left_reg, y)[0]
yhat = left_reg @ left_coef
ydiff = y - yhat

In [58]:
right_reg = f 
right_coef = jnp.linalg.lstsq(right_reg, d)[0]
dhat = right_reg @ right_coef
diff = d - dhat

In [60]:
#final_regs = #jnp.hstack((diff*t, diff))
beta = jnp.linalg.lstsq(diff, ydiff)[0]

In [61]:
beta

DeviceArray([[1.3994378e-06]], dtype=float32)